Homework 3: Sentiment Analysis
----

The following instructions apply to all notebooks and `.py` files you submit for this homework.

Due date: April 15th, 2024 11:59 PM (EST)

Total Points: (105)
- Task 0: 05 points
- Task 1: 10 points
- Task 2: 20 points
- Task 3: 25 points
- Task 4: 40 points (question in LSTM_EncDec.ipynb)

Goals:
- understand the difficulties of counting and probabilities in NLP applications
- work with real world data using different approaches to classification
- stress test your model (to some extent)


Allowed python modules:
- `numpy`, `matplotlib`, `keras`, `pytorch`, `nltk`, `pandas`, `sci-kit learn` (`sklearn`), `seaborn`, and all built-in python libraries (e.g. `math` and `string`)
- if you would like to use a library not on this list, please check with us on Campuswire first.
- all *necessary* imports have been included for you (all imports that we used in our solution)

Instructions:
- Complete outlined problems in this notebook.
- When you have finished, __clear the kernel__ and __run__ your notebook "fresh" from top to bottom. Ensure that there are __no errors__.
    - If a problem asks for you to write code that does result in an error (as in, the answer to the problem is an error), leave the code in your notebook but commented out so that running from top to bottom does not result in any errors.
- Double check that you have completed Task 0.
- Submit your work on Gradescope.
- Double check that your submission on Gradescope looks like you believe it should.

Names & Sections
----
Names: __Sudarshan Paranjape__

Task 0: Name, References, Reflection (5 points)
---

References
---
List the resources you consulted to complete this homework here. Write one sentence per resource about what it provided to you. If you consulted no references to complete your assignment, write a brief sentence stating that this is the case and why it was the case for you.

(Example)
- https://docs.python.org/3/tutorial/datastructures.html
    - Read about the the basics and syntax for data structures in python.

AI Collaboration
---
Following the *Policy on the use of Generative AI* in the syllabus, please cite any LLMs that you used here and briefly describe what you used them for, including to improve language clarity in the written sections.

Reflection
----
Answer the following questions __after__ you complete this assignment (no more than 1 sentence per question required, this section is graded on completion):

1. Does this work reflect your best effort? -- yes
2. What was/were the most challenging part(s) of the assignment? -- Finetuning the TFIDF and FFNN
3. If you want feedback, what function(s) or problem(s) would you like feedback on and why?
4. Briefly reflect on how your partnership functioned--who did which tasks, how was the workload on each of you individually as compared to the previous homeworks, etc.

Task 1: Provided Data Write-Up (10 points)
---

Every time you use a data set in an NLP application (or in any software application), you should be able to answer a set of questions about that data. Answer these now. Default to no more than 1 sentence per question needed. If more explanation is necessary, do give it.

This is about the __provided__ movie review data set.

1. Where did you get the data from? The provided dataset(s) were sub-sampled from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
2. (1 pt) How was the data collected (where did the people acquiring the data get it from and how)? -- Data was collected by scraping the IMDB website<br>

3. (2 pts) How large is the dataset (answer for both the train and the dev set, separately)? (# reviews, # tokens in both the train and dev sets) -- Train dataset has 1600 reviews and ~22000 tokens. Test dataset has 200 reviews and similar no. of tokens.<br>
<br>

4. (1 pt) What is your data? (i.e. newswire, tweets, books, blogs, etc) -- the data is 50K movie reviews having 25,000 highly polar movie reviews for training and 25,000 for testing<br>
<br>
5. (1 pt) Who produced the data? (who were the authors of the text? Your answer might be a specific person or a particular group of people) -- The data has been created by Stanford University<br>
<br>
6. (2 pts) What is the distribution of labels in the data (answer for both the train and the dev set, separately)? -- for the training set nos 1 = 804, 0 = 796<br>for testing set nos 1 = 109, 0 = 91
<br>
7. (2 pts) How large is the vocabulary (answer for both the train and the dev set, separately)? train_set = 30705, dev_set = 8953


8. (1 pt) How big is the overlap between the vocabulary for the train and dev set? -- 6574

In [56]:
train_data_file = 'movie_reviews_train.txt'
train_df = pd.read_csv(train_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_train, y_train = train_df['review'].values, train_df['label'].values

dev_data_file = 'movie_reviews_dev.txt'
dev_df = pd.read_csv(dev_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_dev, y_dev = dev_df['review'].values, dev_df['label'].values

test_data_file = 'movie_reviews_test.txt'
test_df = pd.read_csv(test_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_test, y_test = test_df['review'].values, test_df['label'].values

from nltk.tokenize import word_tokenize
def num_tokens(data):
    vocab = set()
    num_tokens = 0
    for doc in data:
        tokens = word_tokenize(doc)
        num_tokens += len(tokens)
        vocab.update(tokens)
    return num_tokens, len(vocab), vocab

train_num_tokens, train_num_vocab, train_vocab = num_tokens(X_train)
dev_num_tokens, dev_num_vocab, dev_vocab = num_tokens(X_dev)
print('Number of tokens in train data:', train_num_tokens)
print('Number of tokens in dev data:', dev_num_tokens)
print('Number of vocab in train data:', train_num_vocab)
print('Number of vocab in dev data:', dev_num_vocab)

print('Overlap between train and dev vocab:', len(train_vocab & dev_vocab))

Number of tokens in train data: 425345
Number of tokens in dev data: 54599
Number of vocab in train data: 30705
Number of vocab in dev data: 8953
Overlap between train and dev vocab: 6574


Task 2: Train a Logistic Regression Model (20 points)
----
1. Implement a custom function to read in a dataset, and return a list of tuples, using the Tf-Idf feature extraction technique.
2. Compare your implementation to `sklearn`'s TfidfVectorizer (imported below) by timing both on the provided datasets using the time module.
3. Using each set of features, and `sklearn`'s implementation of `LogisticRegression`, train a machine learning model to predict sentiment on the given dataset.

In [33]:
import nltk
#nltk.download('punkt')
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from collections import Counter
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#nltk.download('stopwords')
stopwords = stopwords.words('english')

In [34]:
from re import U
# The following function reads a data-file and splits the contents by tabs.
# The first column is an ID, and thus is discarded. The second column consists of the actual reviews data.
# The third column is the true label for each data point.

# The function returns two objects - a list of all reviews, and a numpy array of labels.
# You will need to use this function later.

def get_lists(input_file):
    f=open(input_file, 'r')
    lines = [line.split('\t')[1:] for line in f.readlines()]
    X = [row[0] for row in lines]
    y=np.array([int(row[1]) for row in lines])
    return X, y

# Fill in the following function to take a corpus (list of reviews) as input,
# extract TfIdf values and return an array of features and the vocabulary.

# If the vocabulary argument is supplied, then the function should only convert the input corpus
# to feature vectors using the provided vocabulary and the max_features argument (if not None).
# In this case, the function should return feature vectors and the supplied vocabulary.

# If the max_features parameter is set to None, then all words in the corpus should be used.
# If the max_features parameter is specified (say, k),
# then only use the k most frequent words in the corpus to build your vocabulary.

# The function should return two things.

# The first object should be a numpy array of shape (n_documents, vocab_size),
# which contains the TF-IDF feature vectors for each document.

# The second object should be a dictionary of the words in the vocabulary,
# mapped to their corresponding index in alphabetical sorted order.
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np

def count_word_occurrences(corpus):
    all_words = []
    for doc in corpus:
        words = word_tokenize(doc.lower())
        all_words.extend(words)
    word_counts = Counter(all_words)
    # Remove stopwords and non-alphabetic tokens
    word_list = [word for word in word_counts if word.isalpha() and word not in stopwords]
    return word_list

def tf_mtx(corpus, unique_words):
    tf_matrix = []
    for doc in corpus:
        doc_tokens = word_tokenize(doc.lower())
        token_freq = Counter(doc_tokens)
        doc_tf = [token_freq[word] / len(doc_tokens) for word in unique_words]
        tf_matrix.append(doc_tf)
    return np.array(tf_matrix)


def idf_mtx(corpus, unique_words, smoothing=1):
    total_docs = len(corpus)
    doc_freq = np.array([sum(1 for doc in corpus if word in doc.split()) for word in unique_words])
    idf_values = np.log((total_docs + smoothing) / (1 + doc_freq))
    idf_dict = dict(zip(unique_words, idf_values))
    return idf_dict


def tf_idf(corpus, unique_words, smoothing=1):
    tf_matrix = tf_mtx(corpus, unique_words)
    idf_dict = idf_mtx(corpus, unique_words, smoothing)
    idf_vector = np.array([idf_dict[word] for word in unique_words])
    
    # Element-wise multiplication of TF matrix with IDF vector
    tf_idf_matrix = tf_matrix * idf_vector
    return tf_idf_matrix

def get_tfidf_vectors(token_lists, max_features=None, vocabulary=None,smoothing = 1):
    #YOUR CODE HERE
    # to consider vocabulary arguement
    if vocabulary is None:
      Uwords = count_word_occurrences(token_lists)
    else:
      Uwords = vocabulary

    vocab_dict = {word: index for index, word in enumerate(sorted(Uwords))}
    # to consider max features arguement
    if max_features is None:
        Uwords
    else:
        # If max_features is specified, get the most frequent words
        Uwords = Uwords[:max_features]

    return tf_idf(token_lists,Uwords,smoothing), vocab_dict

In [35]:
unique_words = ['word1', 'word2', 'word3']  # Example list of unique words
word_counts = dict.fromkeys(unique_words, 0)
word_counts

{'word1': 0, 'word2': 0, 'word3': 0}

In [36]:
X, y = get_lists('movie_reviews_train.txt')

We will now compare the runtime of our Tf-Idf implementation to the `sklearn` implementation. Call the respective functions with appropriate arguments in the code block below.

In [37]:
TRAIN_FILE = "movie_reviews_train.txt"
TEST_FILE = "movie_reviews_test.txt"

train_corpus, y_train = get_lists(TRAIN_FILE)
test_corpus, y_test = get_lists(TEST_FILE)

In [39]:
%%time
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
TEST_FILE = "movie_reviews_test.txt"

train_corpus, y_train = get_lists(TRAIN_FILE)

# First we will use our custom vectorizer to convert words to features, and time it.
###### YOUR CODE HERE #######

train_matrix, vocab = get_tfidf_vectors(train_corpus)

print(train_matrix.shape)
# print("Custom--- Time taken: ", end-start, " seconds")

# Next we will use sklearn's TfidfVectorizer to load in the data, and time it.

(1600, 21430)
CPU times: total: 7min 57s
Wall time: 7min 57s


In [40]:
%%time
test_matrix,_ = get_tfidf_vectors(test_corpus, vocabulary=vocab)
print(test_matrix.shape)

(200, 21430)
CPU times: total: 1min 4s
Wall time: 1min 5s


In [41]:
%%time
vec = TfidfVectorizer()
train_matrix_sk = vec.fit_transform(train_corpus)
print(train_matrix_sk.shape)


test_matrix_sk = vec.transform(test_corpus)
print(test_matrix_sk.shape)

(1600, 22684)
(200, 22684)
CPU times: total: 344 ms
Wall time: 433 ms


NOTE: Ideally, your vectorizer should be within one order of magnitude of the sklearn implementation.

In [42]:
# Any additional code needed to answer questions below
sparsity = np.mean(train_matrix == 0, axis=1) * 100
mean_custom = np.mean(sparsity)
mean_custom

99.56291122258517

In [43]:
sparsity = np.mean((train_matrix_sk.toarray()) == 0, axis =1 )*100
mean_sklearn = np.mean(sparsity)
mean_sklearn

99.39492318374185

1. How large is the vocabulary generated by your vectorizer?<br> **21430**
2. How large is the vocabulary generated by the `sklearn` TfidfVectorizer?<br> **22684**
3. Where might these differences be coming from?<br> **Preprocessing. Stemming and tokenising cause a lot of difference in word count for both the vocabularies**
4. What steps did you take to ensure your vectorizer is optimized for best possible runtime?<br> **Tried to reduce the no. of for loops required to get the vectors**
5. How sparse are your custom features (average percentage of features per review that are zero)?<br> **99.56**
6. How sparse are the TfidfVectorizer's features?<br> **99.39**

NOTE: if you set the lowercase option to False, the sklearn vectorizer should have a vocabulary of around 50k words/tokens.

In [44]:
%%time
vec = TfidfVectorizer(lowercase=False)
train_matrix_sk = vec.fit_transform(train_corpus)
print(train_matrix_sk.shape)


test_matrix_sk = vec.transform(test_corpus)
print(test_matrix_sk.shape)

(1600, 26373)
(200, 26373)
CPU times: total: 297 ms
Wall time: 303 ms


**Logistic Regression**

Now, we will compare how our custom features stack up against sklearn's TfidfVectorizer, by training two separate Logistic Regression classifiers - one on each set of feature vectors. Then load the test set, and convert it to two sets of feature vectors, one using our custom vectorizer (to do this, provide the vocabulary as a function argument), and one using sklearn's Tfidf (use the same object as before to transform the test inputs). For both classifiers, print the average accuracy on the test set and the F1 score.

In [45]:
print(train_matrix.shape)
print(test_matrix.shape)
print(train_matrix_sk.shape)
print(test_matrix_sk.shape)

(1600, 21430)
(200, 21430)
(1600, 26373)
(200, 26373)


In [46]:
# First use sklearn's LogisticRegression classifier to do sentiment analysis using your custom feature vectors:

###### YOUR CODE HERE #######
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Train the Model
print("CUSTOM Model")
model = LogisticRegression()
# print(train_matrix.shape)
model.fit(train_matrix, y_train)

# Load the test data, extract features using your custom vectorizer, and test the performance of the LR classifier
###### YOUR CODE HERE #######

y_pred_test = model.predict(test_matrix)

# Print the accuracy of your model on the test data
###### YOUR CODE HERE #######

test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)

score = f1_score(y_test,y_pred_test)
print("F1 score", score)

# Now repeat the above steps, but this time using features extracted by sklearn's Tfidfvectorizer
print()
print('*'*100)
print()
###### YOUR CODE HERE #######
print("SKLEARN Model")
# print(test_matrix_sk.shape)
# print(sklearn_train_matrix.shape)

skmodel = LogisticRegression()
skmodel.fit(train_matrix_sk, y_train)

y_pred_test_sk = skmodel.predict(test_matrix_sk)

test_accuracy_sk = accuracy_score(y_test, y_pred_test_sk)
print("Test Accuracy:", test_accuracy_sk)

score_sk = f1_score(y_test,y_pred_test_sk)
print("F1 score", score_sk)
    

CUSTOM Model
Test Accuracy: 0.54
F1 score 0.6953642384105961

****************************************************************************************************

SKLEARN Model
Test Accuracy: 0.785
F1 score 0.8018433179723502


NOTE: we're expecting to see a F1 score of around 80% using both your custom features and the sklearn features.

Finally, repeat the process (training and testing), but this time, set the max_features argument to 1000 for both our custom vectorizer and sklearn's Tfidfvectorizer. Report average accuracy and F1 scores for both classifiers.

In [47]:
%%time
###### YOUR CODE HERE #######
print("CUSTOM")
train_matrix_1000, vocab = get_tfidf_vectors(train_corpus, max_features=1000)
print(train_matrix_1000.shape)

test_matrix_1000,_ = get_tfidf_vectors(test_corpus, vocabulary=list(vocab.keys()),max_features=1000)
print(test_matrix_1000.shape)

print('Sklearn')
vec = TfidfVectorizer(max_features=1000)
train_matrix_sk_1000 = vec.fit_transform(train_corpus)
print(train_matrix_sk_1000.shape)
test_matrix_sk_1000 = vec.transform(test_corpus)
print(test_matrix_sk_1000.shape)


model = LogisticRegression()
# print(train_matrix.shape)
model.fit(train_matrix_1000, y_train)
print()
print()
# Load the test data, extract features using your custom vectorizer, and test the performance of the LR classifier
###### YOUR CODE HERE #######

y_pred_test = model.predict(test_matrix_1000)

# Print the accuracy of your model on the test data
###### YOUR CODE HERE #######
print('Custom TFIDF Model')
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)

score = f1_score(y_test,y_pred_test)
print("F1 score", score)

# Now repeat the above steps, but this time using features extracted by sklearn's Tfidfvectorizer
print()
print()
###### YOUR CODE HERE #######
print("Sklearn TFIDF Model")
# print(test_matrix_sk.shape)
# print(sklearn_train_matrix.shape)

skmodel = LogisticRegression()
skmodel.fit(train_matrix_sk_1000, y_train)

y_pred_test_sk = skmodel.predict(test_matrix_sk_1000)

test_accuracy_sk = accuracy_score(y_test, y_pred_test_sk)
print("Test Accuracy:", test_accuracy_sk)

score_sk = f1_score(y_test,y_pred_test_sk)
print("F1 score", score_sk)


CUSTOM
(1600, 1000)
(200, 1000)
Sklearn
(1600, 1000)
(200, 1000)


Custom TFIDF Model
Test Accuracy: 0.55
F1 score 0.7077922077922079


Sklearn TFIDF Model
Test Accuracy: 0.785
F1 score 0.8036529680365296
CPU times: total: 31.7 s
Wall time: 31.6 s


1. Is there a stark difference between the two vectorizers with 1000 features?<br>**There is a difference between the Accuracy and the F1 scored**

2. Use sklearn's documentation for the Tfidfvectorizer to figure out what may be causing the performance difference (or lack thereof).<br>**performance is likely because TFIDF vectorizers are good at identifying and using the top 1000 most important words for sentiment analysis.**

NOTE: Irrespective of your conclusions, both implementations should be above 60% F1 Score.

Task 3: Train a Feedforward Neural Network Model (25 points)
----
1. Using PyTorch, implement a feedforward neural network to do sentiment analysis. This model should take sparse vectors of length 10000 as input (note this is 10000, not 1000), and have a single output with the sigmoid activation function. The number of hidden layers, and intermediate activation choices are up to you, but please make sure your model does not take more than ~1 minute to train.
2. Evaluate the model using PyTorch functions for average accuracy, area under the ROC curve and F1 scores (see [torcheval](https://pytorch.org/torcheval/stable/)) using both vectorizers, with max_features set to 10000 in both cases.

In [48]:
! pip install dataloader

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim


if torch.backends.mps.is_available():
	device = torch.device("mps")
elif torch.cuda.is_available():
	device = torch.device("cuda")
else:
	device = torch.device("cpu")


In [50]:
device

device(type='cpu')

In [51]:
class feedforward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10000, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = torch.sigmoid(self.fc3(X))
        return X

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            return self(X)

In [52]:
# Load the data using custom and sklearn vectors
train_corpus, y_train = get_lists(TRAIN_FILE)
test_corpus, y_test = get_lists(TEST_FILE)

vectorizer = TfidfVectorizer(max_features=10000)
X_train_sklearn = vectorizer.fit_transform(train_corpus)
X_test_sklearn = vectorizer.transform(test_corpus)

X_train_custom, vocab_custom = get_tfidf_vectors(train_corpus, max_features=10000)
X_test_custom, _ = get_tfidf_vectors(test_corpus, vocabulary=vocab_custom)

In [53]:
# Train the model for 50 epochs on both custom and sklearn vectors

# Function for training and evaluating the model
def train(X_train, y_train):
    X_train_torch = torch.tensor(X_train).float().to(device)
    y_train_torch = torch.tensor(y_train).float().to(device)

    # Create a DataLoader for the training data
    train_data = TensorDataset(X_train_torch, y_train_torch)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    # Create a feedforward neural network model
    # you may use any activation function on the hidden layers
    # you should use binary cross-entropy as your loss function
    # Adam is an appropriate optimizer for this task
    model = feedforward().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())

    # Train the model
    for epoch in range(50):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/50, Loss: {loss.item()}')

    return model

print("Training model with custom features")
custom_model = train(X_train_custom, y_train)

print("\nTraining model with sklearn features")
sklearn_model = train(X_train_sklearn.toarray(), y_train)

Training model with custom features
Epoch 1/50, Loss: 0.6868590116500854
Epoch 2/50, Loss: 0.6446149349212646
Epoch 3/50, Loss: 0.49521809816360474
Epoch 4/50, Loss: 0.24817170202732086
Epoch 5/50, Loss: 0.0795406773686409
Epoch 6/50, Loss: 0.03856183961033821
Epoch 7/50, Loss: 0.027150766924023628
Epoch 8/50, Loss: 0.009273674339056015
Epoch 9/50, Loss: 0.012177294120192528
Epoch 10/50, Loss: 0.006316666956990957
Epoch 11/50, Loss: 0.0064244200475513935
Epoch 12/50, Loss: 0.0033264700323343277
Epoch 13/50, Loss: 0.003327655838802457
Epoch 14/50, Loss: 0.0030857669189572334
Epoch 15/50, Loss: 0.002212689258158207
Epoch 16/50, Loss: 0.0018345618154853582
Epoch 17/50, Loss: 0.0017266294453293085
Epoch 18/50, Loss: 0.0014591044746339321
Epoch 19/50, Loss: 0.0011673301924020052
Epoch 20/50, Loss: 0.000984447542577982
Epoch 21/50, Loss: 0.0008860075031407177
Epoch 22/50, Loss: 0.0007502841181121767
Epoch 23/50, Loss: 0.0009571277187205851
Epoch 24/50, Loss: 0.0006659355130977929
Epoch 25/50

In [54]:
!pip install torcheval

# Evaluate the model using custom and sklearn vectors
# Test the model using custom and sklearn vectors
# Evaluate the model and report the score using Binary F1 score, Binary AUROC and Binary accuracy

from torcheval.metrics import BinaryAccuracy, BinaryAUROC, BinaryF1Score

accuracy_metric = BinaryAccuracy()
auroc_metric = BinaryAUROC()
f1_metric = BinaryF1Score()

# Function for evaluating the model
def evaluate_model(model, X_test, y_test):
    X_test_torch = torch.tensor(X_test).float().to(device)
    y_test_torch = torch.tensor(y_test).float().to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_torch).squeeze()

    # Update metric calculations
    accuracy_metric.update(outputs, y_test_torch)
    auroc_metric.update(outputs, y_test_torch)
    f1_metric.update(outputs, y_test_torch)

    # Compute final metrics
    final_accuracy = accuracy_metric.compute()
    final_auroc = auroc_metric.compute()
    final_f1 = f1_metric.compute()

    accuracy_metric.reset()
    auroc_metric.reset()
    f1_metric.reset()

    return final_accuracy.item(), final_auroc.item(), final_f1.item()


print("\nSklearn TfidfVectorizer Model:")
print("Accuracy:", sklearn_accuracy)
print("AUROC:", sklearn_auroc)
print("F1 Score:", sklearn_f1)


Sklearn TfidfVectorizer Model:
Accuracy: 0.8149999976158142
AUROC: 0.8978727694324025
F1 Score: 0.8195121884346008


In [ ]:
custom_accuracy, custom_auroc, custom_f1 = evaluate_model(custom_model, X_test_custom, y_test)

print("Custom TF-IDF Model:")
print("Accuracy:", custom_accuracy)
print("AUROC:", custom_auroc)
print("F1 Score:", custom_f1)

NOTE: As in the last task, we're expecting to see a F1 score of over 60% using both your custom features and the sklearn features.

5 points in this assignment are reserved for overall style (both for writing and for code submitted). All work submitted should be clear, easily interpretable, and checked for spelling, etc. (Re-read what you write and make sure it makes sense). Course staff are always happy to give grammatical help (but we won't pre-grade the content of your answers).